In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
sys.path.append("/home/files/feature_orthogonality")
from utils.session_config import setup_gpus

In [2]:
import tensorflow as tf
keras = tf.keras
import cv2
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"
setup_gpus(memory_fraction=0.2)

1 Physical GPUs, 1 Logical GPUs


In [4]:
from tensorflow.keras.layers import InputLayer, Conv2D, MaxPool2D, Flatten, Dense, ReLU, BatchNormalization, GlobalAveragePooling2D
from utils.losses import L2_Orthogonal
from utils.BEP import convert_to_separate_perceptrons, Back_etching_propagation
from utils.layers import Etching_Layer

In [5]:
trained_model = tf.keras.models.load_model("mnist-3-cat-classifier.h5")

In [6]:
bep_algorithm = Back_etching_propagation(trained_model)

2 layer name :  conv2d
Conv2D
0 layer name :  re_lu
0 layer name :  max_pooling2d
2 layer name :  conv2d_1
Conv2D
0 layer name :  re_lu_1
0 layer name :  max_pooling2d_1
2 layer name :  conv2d_2
Conv2D
0 layer name :  re_lu_2
0 layer name :  global_average_pooling2d
2 layer name :  dense
Dense


In [7]:
model = bep_algorithm.return_model()
trained_model.summary()
#model.output

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
re_lu (ReLU)                 (None, 26, 26, 16)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 16)        2320      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 11, 11, 16)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 16)          2

# Etching

In [1]:
data_0 = np.load("/home/files/datasets/0.npy")
label_0 = np.zeros(len(data_0), np.int32)
data_5 = np.load("/home/files/datasets/5.npy")
label_5 = np.ones(len(data_5), np.int32)
data_7 = np.load("/home/files/datasets/7.npy")
label_7 = np.ones(len(data_7), np.int32)*2

train_x = tf.concat([data_0, data_5, data_7], axis=0)
train_x = tf.cast(train_x, tf.float32)
train_x = train_x[:,:,:,tf.newaxis]/255.
train_y = tf.concat([label_0, label_5, label_7], axis=0)

print(train_x.shape)
print(train_y.shape)

NameError: name 'np' is not defined

In [9]:
x = tf.data.Dataset.from_tensor_slices(train_x)
y = tf.data.Dataset.from_tensor_slices(train_y)
ds = tf.data.Dataset.zip((x, y))
ds = ds.shuffle(len(train_x), reshuffle_each_iteration=False).batch(32)
#ds = ds.shuffle(len(train_x), reshuffle_each_iteration=False).shuffle(1000).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [10]:
#etching start
bep_algorithm.categorical_etching(ds, num_class=3, save_path='categorical_etching')

17
12
7
2


In [ ]:
bep_algorithm = Back_etching_propagation(trained_model)
data = np.load("mnist/0.npy")[...,np.newaxis]/255.
train_x = tf.data.Dataset.from_tensor_slices(data)
train_x = train_x.batch(32)

for x in train_x.take(1):
    print(x.shape)
bep_algorithm.etching(0,train_x)

In [ ]:
#bep_algorithm = Back_etching_propagation(trained_model)
data = np.load("mnist/5.npy")[...,np.newaxis]/255.
train_x = tf.data.Dataset.from_tensor_slices(data)
train_x = train_x.batch(32)

for x in train_x.take(1):
    print(x.shape)
bep_algorithm.etching(1,train_x)

In [ ]:
#bep_algorithm = Back_etching_propagation(trained_model)
data = np.load("mnist/7.npy")[...,np.newaxis]/255.
train_x = tf.data.Dataset.from_tensor_slices(data)
train_x = train_x.batch(32)

for x in train_x.take(1):
    print(x.shape)
bep_algorithm.etching(2,train_x)